In [8]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import math

In [9]:
tbfis_hinfeletronica = pd.read_csv('tbfis_hinfeletron_not_null.csv', sep = ',')
df = tbfis_hinfeletronica

In [10]:
%%time

df["enderecoCompleto"] = df.apply(lambda row: f"{row['nologradouro']}, {row['nrresidencia']}, {row['municipio']}, {row['sgufhinfeletron']}, BRASIL", axis=1)
enderecos_emitentes = df[["sqhinfeletron","nrcep", "nologradouro", "municipio", "sgufhinfeletron", "enderecoCompleto"]]

enderecos_emitentes['contagemEnderecoCompleto'] = enderecos_emitentes['enderecoCompleto'].map(enderecos_emitentes['enderecoCompleto'].value_counts())
enderecos_emitentes = enderecos_emitentes.sort_values(by='contagemEnderecoCompleto', ascending=False)
enderecos_emitentes.drop_duplicates(subset=['enderecoCompleto'], keep='first', inplace=True)
enderecos_emitentes.reset_index(drop=True, inplace=True)

enderecos_emitentes

CPU times: total: 46.9 ms
Wall time: 75.6 ms


<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto
0,15570529,58310000,ROD.BR 230 KM 12,CABEDELO,PB,"ROD.BR 230 KM 12, 0, CABEDELO, PB, BRASIL",2
1,15570221,9715021,RUA AMERICO BRASILIENSE,SAO BERNARDO DO CAMPO,SP,"RUA AMERICO BRASILIENSE, 596, SAO BERNARDO DO ...",2
2,15570315,86790000,AV GRALHA AZUL,LOBATO,PR,"AV GRALHA AZUL, 1196, LOBATO, PR, BRASIL",2
3,15570114,58706310,LIMA CAMPOS,PATOS,PB,"LIMA CAMPOS, 123, PATOS, PB, BRASIL",2
4,15569702,62800000,RUA CORONEL ALEXANDRINO,ARACATI,CE,"RUA CORONEL ALEXANDRINO, 750, ARACATI, CE, BRASIL",2
...,...,...,...,...,...,...,...
984,15569780,58040250,SAO SEBASTIAO,JOAO PESSOA,PB,"SAO SEBASTIAO, 436, JOAO PESSOA, PB, BRASIL",1
985,15569781,58200000,Rua Carlos Gomes,GUARABIRA,PB,"Rua Carlos Gomes, 125, GUARABIRA, PB, BRASIL",1
986,15569782,58338000,Rua Luis de Holanda,PILAR,PB,"Rua Luis de Holanda, 68, PILAR, PB, BRASIL",1
987,15569783,58880000,RUA MANOEL EMIDIO,BREJO DOS SANTOS,PB,"RUA MANOEL EMIDIO, 179, BREJO DOS SANTOS, PB, ...",1


In [10]:
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from time import sleep
import pandas as pd
from tqdm import tqdm

def get_coordinates_from_address(address):
    geolocator = Nominatim(user_agent="cep_geocoder", timeout=5)
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        return ('NaN', 'NaN')
    except (GeocoderTimedOut, GeocoderServiceError):
        return ('NaN', 'NaN')

def get_coordinates_from_cep(cep, retries=3):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and 'erro' not in response.json():
                address = response.json()
                formatted_address = f"{address.get('logradouro', '')}, {address['localidade']}, {address['uf']}, Brasil"
                return get_coordinates_from_address(formatted_address)
            else:
                return ('NaN', 'NaN')
        except requests.exceptions.RequestException:
            sleep(1)
    return ('NaN', 'NaN')

def apply_coordinates_to_dataframe(df, address_column, cep_column):
    latitudes = []
    longitudes = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        address = row[address_column]
        cep = row[cep_column]
        if pd.notna(address):
            lat, lon = get_coordinates_from_address(address)
        elif pd.notna(cep):
            lat, lon = get_coordinates_from_cep(cep)
        else:
            lat, lon = ('NaN', 'NaN')
        latitudes.append(lat)
        longitudes.append(lon)
    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df



df = apply_coordinates_to_dataframe(enderecos_emitentes[:100], 'enderecoCompleto', 'nrcep')
df


100%|██████████| 101/101 [02:32<00:00,  1.51s/it]
C:\Users\jamil\AppData\Local\Temp\ipykernel_5656\1665758061.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = latitudes
C:\Users\jamil\AppData\Local\Temp\ipykernel_5656\1665758061.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = longitudes


,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude
0,15570529,58310000,ROD.BR 230 KM 12,CABEDELO,PB,"ROD.BR 230 KM 12, 0, CABEDELO, PB, BRASIL",2,NaN,NaN
1,15570221,9715021,RUA AMERICO BRASILIENSE,SAO BERNARDO DO CAMPO,SP,"RUA AMERICO BRASILIENSE, 596, SAO BERNARDO DO ...",2,-23.713083,-46.547259
2,15570315,86790000,AV GRALHA AZUL,LOBATO,PR,"AV GRALHA AZUL, 1196, LOBATO, PR, BRASIL",2,-23.01756,-51.951985
3,15570114,58706310,LIMA CAMPOS,PATOS,PB,"LIMA CAMPOS, 123, PATOS, PB, BRASIL",2,NaN,NaN
4,15569702,62800000,RUA CORONEL ALEXANDRINO,ARACATI,CE,"RUA CORONEL ALEXANDRINO, 750, ARACATI, CE, BRASIL",2,NaN,NaN
...,...,...,...,...,...,...,...,...,...
96,15570216,58890000,R CAPITAO LIRA,BREJO DO CRUZ,PB,"R CAPITAO LIRA, 11, BREJO DO CRUZ, PB, BRASIL",1,-6.34929,-37.4983
97,15570217,58102833,RUA DOM JOSE TOMAZ,CABEDELO,PB,"RUA DOM JOSE TOMAZ, 81, CABEDELO, PB, BRASIL",1,NaN,NaN
98,15570218,58011510,Rua Venâncio Vital da Silva,JOAO PESSOA,PB,"Rua Venâncio Vital da Silva, 16, JOAO PESSOA, ...",1,-7.12945,-34.903228
99,15570219,58045010,ALAMEDA CABO BRANCO 4660 APT 201,JOAO PESSOA,PB,"ALAMEDA CABO BRANCO 4660 APT 201, 201, JOAO PE...",1,NaN,NaN


In [11]:
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from time import sleep
import pandas as pd
from tqdm import tqdm

def get_coordinates_from_address(address):
    geolocator = Nominatim(user_agent="cep_geocoder", timeout=5)
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        return ('NaN', 'NaN')
    except (GeocoderTimedOut, GeocoderServiceError):
        return ('NaN', 'NaN')

def get_coordinates_from_cep(cep, retries=3):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and 'erro' not in response.json():
                address = response.json()
                formatted_address = f"{address.get('logradouro', '')}, {address['localidade']}, {address['uf']}, Brasil"
                return get_coordinates_from_address(formatted_address)
            else:
                return ('NaN', 'NaN')
        except requests.exceptions.RequestException:
            sleep(1)
    return ('NaN', 'NaN')

def apply_coordinates_to_dataframe(df, address_column, cep_column):
    latitudes = []
    longitudes = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        address = row[address_column]
        cep = row[cep_column]
        if pd.notna(address):
            lat, lon = get_coordinates_from_address(address)
        elif pd.notna(cep):
            lat, lon = get_coordinates_from_cep(cep)
        else:
            lat, lon = ('NaN', 'NaN')
        latitudes.append(lat)
        longitudes.append(lon)
    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df

enderecos_emitentes_filtrado = enderecos_emitentes[enderecos_emitentes['sqhinfeletron'] == 15570114]


df = apply_coordinates_to_dataframe(enderecos_emitentes_filtrado, 'enderecoCompleto', 'nrcep')
df

100%|██████████| 1/1 [00:05<00:00,  5.25s/it]
C:\Users\jamil\AppData\Local\Temp\ipykernel_17136\533060819.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = latitudes
C:\Users\jamil\AppData\Local\Temp\ipykernel_17136\533060819.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = longitudes


,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude
3,15570114,58706310,LIMA CAMPOS,PATOS,PB,"LIMA CAMPOS, 123, PATOS, PB, BRASIL",2,NaN,NaN


In [6]:
#coordenadas otimizadas

import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from time import sleep
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_coordinates_from_address(address):
    geolocator = Nominatim(user_agent="cep_geocoder", timeout=5)
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        return ('NaN', 'NaN')
    except (GeocoderTimedOut, GeocoderServiceError):
        return ('NaN', 'NaN')

def get_coordinates_from_cep(cep):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200 and 'erro' not in response.json():
            address = response.json()
            formatted_address = f"{address.get('logradouro', '')}, {address['localidade']}, {address['uf']}, Brasil"
            return get_coordinates_from_address(formatted_address)
        else:
            return ('NaN', 'NaN')
    except requests.exceptions.RequestException:
        return ('NaN', 'NaN')

def process_row(row):
    address = row['enderecoCompleto']
    cep = row['nrcep']
    if pd.notna(address):
        return get_coordinates_from_address(address)
    elif pd.notna(cep):
        return get_coordinates_from_cep(cep)
    else:
        return ('NaN', 'NaN')

def apply_coordinates_to_dataframe(df):
    results = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(process_row, row) for _, row in tqdm(df.iterrows(), total=df.shape[0])]
        for future in as_completed(futures):
            results.append(future.result())

    df['latitude'], df['longitude'] = zip(*results)
    return df

df = apply_coordinates_to_dataframe(enderecos_emitentes)



100%|██████████| 989/989 [00:01<00:00, 495.57it/s] 


In [5]:
df.query('sqhinfeletron == 15569418')
#df['Latitude'].value_counts().get('NaN', 0)

,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude


In [7]:
coordenadas = df[["sqhinfeletron", "latitude", "longitude"]]
coordenadas

,sqhinfeletron,latitude,longitude
0,15570529,-23.713083,-46.547259
1,15570221,NaN,NaN
2,15570315,-7.165523,-34.829576
3,15570114,NaN,NaN
4,15569702,NaN,NaN
...,...,...,...
984,15569780,NaN,NaN
985,15569781,NaN,NaN
986,15569782,NaN,NaN
987,15569783,NaN,NaN


In [55]:
tbfis_hinfeletronica_completo = pd.merge(tbfis_hinfeletronica, coordenadas, on='sqhinfeletron', how='left', suffixes=('', '_lista_enderecos'))
tbfis_hinfeletronica_completo

,sqhinfeletron,sqhumanoinst,municipio,sqtpdochuminst,cdpais,nrcep,nrdocumento,nopais,idestrangeiro,dscomplemento,...,noemail,norazaosocial,nrinscricao,nrinscrsuframa,nrresidencia,nrtelefone,sgufhinfeletron,tsatualizacao,latitude,longitude
0,15569417,11783220,PARNAMIRIM,3,NaN,59152505,10446336467,Brasil,NaN,NaN,...,NaN,JULIA CRISTINA M C DA SILVA,NaN,NaN,69,NaN,RN,2016-09-21 06:12:29.000,-5.891482,-35.205362
1,15569418,11783221,NATAL,3,NaN,59074060,5889831445,Brasil,NaN,NaN,...,NaN,ROSIANE ROCHA RAFAEL,NaN,NaN,0,NaN,RN,2016-09-21 06:13:23.000,-5.821089,-35.256911
2,15569419,11783222,MACAIBA,3,NaN,59280000,9907707457,Brasil,NaN,NaN,...,NaN,ALINE DANIELLE MENDES BEZERRA,NaN,NaN,19,NaN,RN,2016-09-21 06:13:25.000,-5.85354,-35.359194
3,15569420,11783223,NATAL,3,NaN,59132690,3363205406,Brasil,NaN,NaN,...,NaN,AURICELIA MARIA DE ALBURQUERQUE,NaN,NaN,11,NaN,RN,2016-09-21 06:13:27.000,-5.716233,-35.237723
4,15569421,8651723,NATAL,3,NaN,59054590,2150325438,Brasil,NaN,NaN,...,NaN,KALIANE KEILLY BRITO R DA SILVA,NaN,NaN,2448,NaN,RN,2016-09-21 06:13:28.000,-5.810005,-35.21256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15570577,11783614,SAO PAULO,3,NaN,4854270,40379455846,BRASIL,NaN,NaN,...,NaN,JUCEILMA GREGORIO VALENA DA SILVA,NaN,NaN,16,NaN,SP,2016-09-21 09:28:21.000,-23.517737,-47.131708
996,15570578,511190,CAMPINA GRANDE,3,NaN,58100001,6603521091,BRASIL,NaN,S/N,...,NaN,WILLIAM IMPERIANO DE CRISTO,NaN,NaN,S/N,NaN,PB,2016-09-21 09:28:22.000,-7.236875,-35.923485
997,15570581,9430452,JOAO PESSOA,3,NaN,58000000,6663272462,BRASIL,NaN,NaN,...,NaN,NILSON DE ANDRADE QUIRINO,NaN,NaN,SN,NaN,PB,2016-09-21 09:28:37.000,NaN,NaN
998,15570583,11738183,JUAZEIRINHO,2,NaN,58660000,25460331000135,BRASIL,NaN,NaN,...,NaN,ELEICAO 2016 SANDRA MARIA PAULINO AMARO VEREADOR,NaN,NaN,S/N,NaN,PB,2016-09-21 09:29:00.000,NaN,NaN


In [56]:
tbfis_hinfeletronica_completo.sort_values(by='sqhinfeletron', ignore_index=True)

,sqhinfeletron,sqhumanoinst,municipio,sqtpdochuminst,cdpais,nrcep,nrdocumento,nopais,idestrangeiro,dscomplemento,...,noemail,norazaosocial,nrinscricao,nrinscrsuframa,nrresidencia,nrtelefone,sgufhinfeletron,tsatualizacao,latitude,longitude
0,15569417,11783220,PARNAMIRIM,3,NaN,59152505,10446336467,Brasil,NaN,NaN,...,NaN,JULIA CRISTINA M C DA SILVA,NaN,NaN,69,NaN,RN,2016-09-21 06:12:29.000,-5.891482,-35.205362
1,15569418,11783221,NATAL,3,NaN,59074060,5889831445,Brasil,NaN,NaN,...,NaN,ROSIANE ROCHA RAFAEL,NaN,NaN,0,NaN,RN,2016-09-21 06:13:23.000,-5.821089,-35.256911
2,15569419,11783222,MACAIBA,3,NaN,59280000,9907707457,Brasil,NaN,NaN,...,NaN,ALINE DANIELLE MENDES BEZERRA,NaN,NaN,19,NaN,RN,2016-09-21 06:13:25.000,-5.85354,-35.359194
3,15569420,11783223,NATAL,3,NaN,59132690,3363205406,Brasil,NaN,NaN,...,NaN,AURICELIA MARIA DE ALBURQUERQUE,NaN,NaN,11,NaN,RN,2016-09-21 06:13:27.000,-5.716233,-35.237723
4,15569421,8651723,NATAL,3,NaN,59054590,2150325438,Brasil,NaN,NaN,...,NaN,KALIANE KEILLY BRITO R DA SILVA,NaN,NaN,2448,NaN,RN,2016-09-21 06:13:28.000,-5.810005,-35.21256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15570577,11783614,SAO PAULO,3,NaN,4854270,40379455846,BRASIL,NaN,NaN,...,NaN,JUCEILMA GREGORIO VALENA DA SILVA,NaN,NaN,16,NaN,SP,2016-09-21 09:28:21.000,-23.517737,-47.131708
996,15570578,511190,CAMPINA GRANDE,3,NaN,58100001,6603521091,BRASIL,NaN,S/N,...,NaN,WILLIAM IMPERIANO DE CRISTO,NaN,NaN,S/N,NaN,PB,2016-09-21 09:28:22.000,-7.236875,-35.923485
997,15570581,9430452,JOAO PESSOA,3,NaN,58000000,6663272462,BRASIL,NaN,NaN,...,NaN,NILSON DE ANDRADE QUIRINO,NaN,NaN,SN,NaN,PB,2016-09-21 09:28:37.000,NaN,NaN
998,15570583,11738183,JUAZEIRINHO,2,NaN,58660000,25460331000135,BRASIL,NaN,NaN,...,NaN,ELEICAO 2016 SANDRA MARIA PAULINO AMARO VEREADOR,NaN,NaN,S/N,NaN,PB,2016-09-21 09:29:00.000,NaN,NaN


In [10]:
enderecos_emitentes.sort_values(by='sqhinfeletron', ignore_index=True)

,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude
0,15569417,59152505,R UMARIZAL,PARNAMIRIM,RN,"R UMARIZAL, 69, PARNAMIRIM, RN, BRASIL",1,-5.891482,-35.205362
1,15569418,59074060,R POTIGUARES,NATAL,RN,"R POTIGUARES, 0, NATAL, RN, BRASIL",1,NaN,NaN
2,15569419,59280000,R MIGUEL DA CRUZ,MACAIBA,RN,"R MIGUEL DA CRUZ, 19, MACAIBA, RN, BRASIL",1,NaN,NaN
3,15569420,59132690,R DOS IMIGRANTES,NATAL,RN,"R DOS IMIGRANTES, 11, NATAL, RN, BRASIL",1,NaN,NaN
4,15569421,59054590,AV BERNARDO VIEIRA,NATAL,RN,"AV BERNARDO VIEIRA, 2448, NATAL, RN, BRASIL",1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
984,15570577,4854270,STO ANTONIO,SAO PAULO,SP,"STO ANTONIO, 16, SAO PAULO, SP, BRASIL",1,NaN,NaN
985,15570578,58100001,AVENIDA MARECHAL FLORIANO PEIXOTO,CAMPINA GRANDE,PB,"AVENIDA MARECHAL FLORIANO PEIXOTO, S/N, CAMPIN...",1,NaN,NaN
986,15570581,58000000,RUA JOAO AMERICO DE CARVALHO SN,JOAO PESSOA,PB,"RUA JOAO AMERICO DE CARVALHO SN, SN, JOAO PESS...",1,NaN,NaN
987,15570583,58660000,RUA RES DISTRITO BARRA DE JUAZEIRINHO,JUAZEIRINHO,PB,"RUA RES DISTRITO BARRA DE JUAZEIRINHO, S/N, JU...",1,NaN,NaN


In [58]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError


df = pd.DataFrame(tbfis_hinfeletronica_completo)

# Configurações de conexão
host = "ceos"
dbname = "teste_espaciais"
user = "postgres"
password = "postgres"
port = "5432"

# Criando o engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Função para converter latitude e longitude em formato GEOMETRY
def convert_to_geometry(lat, lon):
    return f'SRID=4326;POINT({lon} {lat})'

# Adicionando coluna de coordenadas ao DataFrame
df['coordenadas'] = df.apply(lambda row: convert_to_geometry(row['latitude'], row['longitude']), axis=1)

# Removendo as colunas de latitude e longitude, pois já temos as coordenadas
#df = df.drop(columns=['enderecoCompleto', 'contagemEnderecoCompleto','Latitude_lista_enderecos','Longitude_lista_enderecos'])
#df = df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude','sqhumanoinst':''})

# Inserindo dados no banco de dados
try:
    df.to_sql('tbfis_hinfeletron', engine, if_exists='append', index=False)
    print("Dados inseridos com sucesso.")
except SQLAlchemyError as e:
    print(f"Erro ao inserir dados: {e}")


Dados inseridos com sucesso.
